In [1]:
from src.drp_fall_2025.fields import PrimeField
from src.drp_fall_2025.linear_algebra import VectorSpace

Proof-of-concept for creating and working with a vector space on $Z/2Z$

In [2]:
Z2 = PrimeField(order=2)

V = VectorSpace(dimension=3, field=Z2)

v1 = V.vector(1, 0, 1)
v2 = V.vector(1, 1, 0)

print(f"Field: {V.field}")
print(f"v1: {v1}")
print(f"v2: {v2}")

# Expected: (1+1, 0+1, 1+1) mod 2 = (0, 1, 1)
v_sum = v1 + v2
print(f"v1 + v2 = {v_sum}")

# Expected: (1*1 + 0*1 + 1*0) mod 2 = (1 + 0 + 0) mod 2 = 1
dot_product = v1.dot(v2)
print(f"v1 . v2 = {dot_product}")

Field: PrimeField(order=2)
v1: Vector(1, 0, 1)
v2: Vector(1, 1, 0)
v1 + v2 = Vector(0, 1, 1)
v1 . v2 = 1
